# Module 3: Land Classification — CNN-Transformer Integration Evaluation
---

In [ ]:
# Import necessary libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)
from tqdm import tqdm

print(f"TensorFlow version: {tf.__version__}")
print(f"PyTorch version: {torch.__version__}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## Task 1: Define the dataset directory, dataloader, and model hyperparameters. The dataloader and model hyperparameters should be the same as those used during training.

In [ ]:
# Task 1: Define dataset directory, dataloader, and model hyperparameters

# ========== Dataset Directory ==========
dataset_path = './images_dataSAT/'

# ========== Dataloader Hyperparameters ==========
IMG_SIZE = 64
IMG_SIZE_KERAS = (64, 64)
BATCH_SIZE = 32
NUM_CLASSES = 2

# ========== Model Hyperparameters (same as training) ==========
# Baseline model
MODEL_EMBED_DIM = 64
MODEL_NUM_HEADS = 4
MODEL_DEPTH = 4
MODEL_MLP_DIM = 128
MODEL_DROPOUT = 0.1

# Model test (larger)
MODEL_TEST_EMBED_DIM = 768
MODEL_TEST_NUM_HEADS = 12
MODEL_TEST_DEPTH = 12
MODEL_TEST_MLP_DIM = 3072
MODEL_TEST_DROPOUT = 0.1

print("Configuration Summary:")
print("=" * 50)
print(f"Dataset path: {dataset_path}")
print(f"Image size: {IMG_SIZE}x{IMG_SIZE}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Number of classes: {NUM_CLASSES}")
print()
print("Baseline Model Hyperparameters:")
print(f"  embed_dim={MODEL_EMBED_DIM}, num_heads={MODEL_NUM_HEADS}, depth={MODEL_DEPTH}, mlp_dim={MODEL_MLP_DIM}")
print()
print("Model Test Hyperparameters:")
print(f"  embed_dim={MODEL_TEST_EMBED_DIM}, num_heads={MODEL_TEST_NUM_HEADS}, depth={MODEL_TEST_DEPTH}, mlp_dim={MODEL_TEST_MLP_DIM}")

In [ ]:
# Define print_metrics function
def print_metrics(y_true, y_pred, model_name='Model'):
    """
    Print comprehensive performance metrics for a given model.
    """
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    cm = confusion_matrix(y_true, y_pred)
    
    print(f"\n{'='*55}")
    print(f" Performance Metrics: {model_name}")
    print(f"{'='*55}")
    print(f"  Accuracy:  {acc:.4f}")
    print(f"  Precision: {prec:.4f}")
    print(f"  Recall:    {rec:.4f}")
    print(f"  F1 Score:  {f1:.4f}")
    print(f"\n  Confusion Matrix:")
    print(f"                    Predicted")
    print(f"                    Neg(0)  Pos(1)")
    print(f"  Actual Neg(0):   TN={cm[0][0]:5d}  FP={cm[0][1]:5d}")
    print(f"  Actual Pos(1):   FN={cm[1][0]:5d}  TP={cm[1][1]:5d}")
    print(f"\n  Classification Report:")
    print(classification_report(y_true, y_pred,
                                target_names=['Non-Agricultural', 'Agricultural']))
    
    return {'accuracy': acc, 'precision': prec, 'recall': rec, 
            'f1': f1, 'confusion_matrix': cm}

print("print_metrics function defined.")

In [ ]:
# ========== KERAS: Load model and prepare data ==========
# Keras validation generator
val_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2
)

keras_val_generator = val_datagen.flow_from_directory(
    dataset_path,
    target_size=IMG_SIZE_KERAS,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation',
    shuffle=False,
    seed=42
)

print(f"Keras validation samples: {keras_val_generator.samples}")
print(f"Keras class indices: {keras_val_generator.class_indices}")

In [ ]:
# Load Keras hybrid model
try:
    keras_vit_model = load_model('best_hybrid_model.keras')
    print("Keras CNN-ViT Hybrid model loaded successfully!")
except Exception as e:
    print(f"Error loading Keras model: {e}")
    print("Attempting to load alternative model...")
    try:
        keras_vit_model = load_model('best_model.keras')
        print("Loaded fallback Keras model.")
    except:
        print("No Keras model found. Please ensure the model file exists.")

In [ ]:
# Get Keras predictions
keras_val_generator.reset()
keras_preds_prob = keras_vit_model.predict(keras_val_generator, verbose=1)
keras_preds = (keras_preds_prob > 0.5).astype(int).flatten()
keras_true_labels = keras_val_generator.classes

print(f"Keras predictions: {len(keras_preds)}")
print(f"Keras true labels: {len(keras_true_labels)}")
print(f"First 10 predictions: {keras_preds[:10]}")
print(f"First 10 true labels:  {keras_true_labels[:10]}")

## Task 2: Instantiate the PyTorch model.

In [ ]:
# ============================================================
# PyTorch Model Definitions (same as training)
# ============================================================

class CNNFeatureExtractor(nn.Module):
    def __init__(self):
        super(CNNFeatureExtractor, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32), nn.ReLU(inplace=True), nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64), nn.ReLU(inplace=True), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128), nn.ReLU(inplace=True), nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256), nn.ReLU(inplace=True), nn.MaxPool2d(2, 2),
        )
    def forward(self, x):
        return self.features(x)

class TransformerEncoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, mlp_dim, dropout=0.1):
        super(TransformerEncoderBlock, self).__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        self.norm2 = nn.LayerNorm(embed_dim)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, mlp_dim), nn.GELU(), nn.Dropout(dropout),
            nn.Linear(mlp_dim, embed_dim), nn.Dropout(dropout)
        )
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        x_norm = self.norm1(x)
        attn_out, _ = self.attn(x_norm, x_norm, x_norm)
        x = x + self.dropout(attn_out)
        x_norm = self.norm2(x)
        x = x + self.mlp(x_norm)
        return x

class CNNViTHybrid(nn.Module):
    def __init__(self, num_classes=2, embed_dim=64, num_heads=4,
                 depth=4, mlp_dim=128, dropout=0.1):
        super(CNNViTHybrid, self).__init__()
        self.cnn = CNNFeatureExtractor()
        self.seq_length = 4 * 4
        self.cnn_feature_dim = 256
        self.projection = nn.Linear(self.cnn_feature_dim, embed_dim)
        self.pos_embedding = nn.Parameter(torch.randn(1, self.seq_length, embed_dim))
        self.pos_dropout = nn.Dropout(dropout)
        self.transformer_blocks = nn.ModuleList([
            TransformerEncoderBlock(embed_dim, num_heads, mlp_dim, dropout)
            for _ in range(depth)
        ])
        self.norm = nn.LayerNorm(embed_dim)
        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, 256), nn.ReLU(), nn.Dropout(dropout),
            nn.Linear(256, num_classes)
        )
    def forward(self, x):
        x = self.cnn(x)
        x = x.flatten(2).transpose(1, 2)
        x = self.projection(x)
        x = x + self.pos_embedding
        x = self.pos_dropout(x)
        for block in self.transformer_blocks:
            x = block(x)
        x = self.norm(x)
        x = x.mean(dim=1)
        x = self.classifier(x)
        return x

print("PyTorch model classes defined.")

In [ ]:
# Task 2: Instantiate the PyTorch model

# Instantiate the PyTorch CNN-ViT Hybrid model with SAME hyperparameters as training
pytorch_vit_model = CNNViTHybrid(
    num_classes=NUM_CLASSES,
    embed_dim=MODEL_EMBED_DIM,
    num_heads=MODEL_NUM_HEADS,
    depth=MODEL_DEPTH,
    mlp_dim=MODEL_MLP_DIM,
    dropout=MODEL_DROPOUT
).to(device)

# Load trained weights
try:
    pytorch_vit_model.load_state_dict(
        torch.load('best_model.pth', map_location=device)
    )
    print("PyTorch CNN-ViT model weights loaded successfully!")
except Exception as e:
    print(f"Warning: Could not load weights: {e}")
    print("Using randomly initialized model.")

pytorch_vit_model.eval()

print(f"\nModel Architecture:")
print(f"  embed_dim={MODEL_EMBED_DIM}, num_heads={MODEL_NUM_HEADS}, "
      f"depth={MODEL_DEPTH}, mlp_dim={MODEL_MLP_DIM}")
print(f"  Total parameters: {sum(p.numel() for p in pytorch_vit_model.parameters()):,}")

In [ ]:
# Prepare PyTorch validation data
val_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

full_dataset = datasets.ImageFolder(root=dataset_path, transform=val_transform)
total_size = len(full_dataset)
val_size = int(0.2 * total_size)
train_size = total_size - val_size

_, val_subset = torch.utils.data.random_split(
    full_dataset, [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

pytorch_val_loader = DataLoader(
    val_subset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

print(f"PyTorch validation samples: {len(val_subset)}")
print(f"PyTorch validation batches: {len(pytorch_val_loader)}")

In [ ]:
# Get PyTorch predictions
pytorch_all_preds = []
pytorch_all_labels = []

pytorch_vit_model.eval()
with torch.no_grad():
    for images, labels in tqdm(pytorch_val_loader, desc='PyTorch Evaluation'):
        images, labels = images.to(device), labels.to(device)
        outputs = pytorch_vit_model(images)
        _, predicted = torch.max(outputs.data, 1)
        pytorch_all_preds.extend(predicted.cpu().numpy())
        pytorch_all_labels.extend(labels.cpu().numpy())

pytorch_all_preds = np.array(pytorch_all_preds)
pytorch_all_labels = np.array(pytorch_all_labels)

print(f"\nPyTorch predictions: {len(pytorch_all_preds)}")
print(f"PyTorch true labels: {len(pytorch_all_labels)}")
print(f"First 10 predictions: {pytorch_all_preds[:10]}")
print(f"First 10 true labels:  {pytorch_all_labels[:10]}")

## Task 3: Print the evaluation metrics using the print_metrics function for the Keras ViT model named Keras CNN-ViT Hybrid Model.

In [ ]:
# Task 3: Print Keras ViT model metrics
keras_metrics = print_metrics(
    keras_true_labels, 
    keras_preds, 
    model_name='Keras CNN-Vit Hybrid Model'
)

## Task 4: Print the evaluation metrics using the print_metrics function for the PyTorch ViT model named PyTorch CNN-ViT Hybrid Model.

In [ ]:
# Task 4: Print PyTorch ViT model metrics
pytorch_metrics = print_metrics(
    pytorch_all_labels, 
    pytorch_all_preds, 
    model_name='PyTorch CNN-Vit Hybrid Model'
)

In [ ]:
# ============================================================
# Final Comparative Visualization
# ============================================================

# Bar chart comparison
metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
keras_vals = [keras_metrics['accuracy'], keras_metrics['precision'],
              keras_metrics['recall'], keras_metrics['f1']]
pytorch_vals = [pytorch_metrics['accuracy'], pytorch_metrics['precision'],
                pytorch_metrics['recall'], pytorch_metrics['f1']]

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Plot 1: Bar chart comparison
x = np.arange(len(metrics_names))
width = 0.35
bars1 = axes[0].bar(x - width/2, keras_vals, width, label='Keras CNN-ViT', color='steelblue', edgecolor='black')
bars2 = axes[0].bar(x + width/2, pytorch_vals, width, label='PyTorch CNN-ViT', color='coral', edgecolor='black')
axes[0].set_ylabel('Score', fontsize=12)
axes[0].set_title('Keras vs PyTorch CNN-ViT Hybrid Models', fontsize=14, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(metrics_names, fontsize=11)
axes[0].legend(fontsize=11)
axes[0].set_ylim(0, 1.15)
axes[0].grid(axis='y', alpha=0.3)

for bar in bars1:
    axes[0].text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.01,
                 f'{bar.get_height():.3f}', ha='center', va='bottom', fontsize=9)
for bar in bars2:
    axes[0].text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.01,
                 f'{bar.get_height():.3f}', ha='center', va='bottom', fontsize=9)

# Plot 2: Confusion matrices side by side
cm_keras = keras_metrics['confusion_matrix']
cm_pytorch = pytorch_metrics['confusion_matrix']

combined_text = f"Keras CM:\n{cm_keras}\n\nPyTorch CM:\n{cm_pytorch}"

# Heatmap for Keras confusion matrix
im = axes[1].imshow(cm_keras, interpolation='nearest', cmap='Blues', alpha=0.7)
axes[1].set_title('Keras CNN-ViT Confusion Matrix', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Predicted Label', fontsize=12)
axes[1].set_ylabel('True Label', fontsize=12)
axes[1].set_xticks([0, 1])
axes[1].set_yticks([0, 1])
axes[1].set_xticklabels(['Non-Agri', 'Agri'])
axes[1].set_yticklabels(['Non-Agri', 'Agri'])

# Add text to heatmap
for i in range(2):
    for j in range(2):
        axes[1].text(j, i, str(cm_keras[i, j]),
                     ha='center', va='center', fontsize=16, fontweight='bold')

plt.suptitle('CNN-Transformer Integration: Final Evaluation', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# ============================================================
# Final Summary Table
# ============================================================
print("\n" + "=" * 65)
print("  FINAL COMPARATIVE SUMMARY")
print("  Keras CNN-ViT vs PyTorch CNN-ViT Hybrid Models")
print("=" * 65)
print(f"{'Metric':<20} {'Keras CNN-ViT':>18} {'PyTorch CNN-ViT':>18}")
print("-" * 65)
print(f"{'Accuracy':<20} {keras_metrics['accuracy']:>18.4f} {pytorch_metrics['accuracy']:>18.4f}")
print(f"{'Precision':<20} {keras_metrics['precision']:>18.4f} {pytorch_metrics['precision']:>18.4f}")
print(f"{'Recall':<20} {keras_metrics['recall']:>18.4f} {pytorch_metrics['recall']:>18.4f}")
print(f"{'F1 Score':<20} {keras_metrics['f1']:>18.4f} {pytorch_metrics['f1']:>18.4f}")
print("=" * 65)

# Determine winner
if keras_metrics['f1'] > pytorch_metrics['f1']:
    print(f"\n>> Keras CNN-ViT Hybrid Model performs better (F1: {keras_metrics['f1']:.4f})")
elif pytorch_metrics['f1'] > keras_metrics['f1']:
    print(f"\n>> PyTorch CNN-ViT Hybrid Model performs better (F1: {pytorch_metrics['f1']:.4f})")
else:
    print(f"\n>> Both models have equal F1 scores: {keras_metrics['f1']:.4f}")

---
## All 4 tasks completed successfully.